In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import joblib
import re
import warnings
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

warnings.filterwarnings('ignore')

print("==================================================")
print("   FOOTBALL MATCH PREDICTOR (PRODUCTION v2.0)     ")
print("==================================================")

# ==========================================
# 1. LOAD & CLEAN DATA
# ==========================================
try:
    df = pd.read_csv("match_data.csv")
except FileNotFoundError:
    print("Error: match_data.csv not found.")
    exit()

# Date Parsing
def extract_date(url):
    try:
        match = re.search(r'([A-Za-z]+-\d{1,2}-\d{4})', str(url))
        if match: return pd.to_datetime(match.group(1), format='%B-%d-%Y', errors='coerce')
    except: pass
    return pd.NaT

df['date'] = df['match_url'].apply(extract_date)
df = df.sort_values(by='date').reset_index(drop=True)

# Numeric Cleaning & Imputation
stats_cols = ["xg", "possession", "shots_onTarget", "corners", "fouls", "team_points"]
for side in ['home', 'away']:
    p_col = f"{side}_team_possession"
    if p_col in df.columns:
        df[p_col] = pd.to_numeric(df[p_col].astype(str).str.rstrip('%'), errors='coerce').fillna(50) / 100.0
    
    for s in stats_cols:
        col = f"{side}_{s}" if s not in ['xg', 'possession', 'team_points'] else f"{side}_team_{s}"
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
            df[col] = df[col].fillna(df.groupby(f'{side}_team_name')[col].transform('median'))
            df[col] = df[col].fillna(df[col].mean())

# ==========================================
# 2. FEATURE ENGINEERING ENGINE
# ==========================================

# --- ELO Calculation ---
def calculate_elo(df):
    k_factor = 20
    home_advantage = 65 
    team_elos = {team: 1500 for team in set(df['home_team_name']).union(set(df['away_team_name']))}
    elo_h, elo_a = [], []
    
    for _, row in df.iterrows():
        h, a = row['home_team_name'], row['away_team_name']
        rh, ra = team_elos[h], team_elos[a]
        elo_h.append(rh); elo_a.append(ra)
        
        if row['home_team_score'] > row['away_team_score']: res = 1
        elif row['home_team_score'] == row['away_team_score']: res = 0.5
        else: res = 0
        
        dr = (rh + home_advantage) - ra
        e_h = 1 / (1 + 10 ** (-dr / 400))
        team_elos[h] = rh + k_factor * (res - e_h)
        team_elos[a] = ra + k_factor * ((1 - res) - (1 - e_h))
        
    return elo_h, elo_a, team_elos

df['home_elo'], df['away_elo'], current_elos = calculate_elo(df)
df['diff_elo'] = (df['home_elo'] + 65) - df['away_elo']

# --- Rest Days ---
long_df = pd.concat([
    df[['date', 'home_team_name']].rename(columns={'home_team_name':'team'}),
    df[['date', 'away_team_name']].rename(columns={'away_team_name':'team'})
]).sort_values(['team', 'date'])
long_df['rest'] = (long_df['date'] - long_df.groupby('team')['date'].shift(1)).dt.days.fillna(7).clip(upper=14)
rest_map = dict(zip(zip(long_df['date'], long_df['team']), long_df['rest']))

df['diff_rest'] = df.apply(lambda x: rest_map.get((x['date'], x['home_team_name']),7), axis=1) - \
                  df.apply(lambda x: rest_map.get((x['date'], x['away_team_name']),7), axis=1)

# --- Rolling Stats ---
# Recalc Points
df['home_team_points'] = np.select([df['home_team_score']>df['away_team_score'], df['home_team_score']==df['away_team_score']], [3, 1], 0)
df['away_team_points'] = np.select([df['away_team_score']>df['home_team_score'], df['away_team_score']==df['home_team_score']], [3, 1], 0)

roll_feats = ['team_xg', 'team_possession', 'shots_onTarget', 'corners', 'team_points']
h_d = df[['date', 'match_url', 'home_team_name']].rename(columns={'home_team_name':'team'})
a_d = df[['date', 'match_url', 'away_team_name']].rename(columns={'away_team_name':'team'})

for f in roll_feats:
    c_h = f"home_{f}" if f in ['team_points', 'team_xg', 'team_possession'] else f"home_{f}"
    c_a = f"away_{f}" if f in ['team_points', 'team_xg', 'team_possession'] else f"away_{f}"
    if c_h in df.columns: h_d[f] = df[c_h]
    if c_a in df.columns: a_d[f] = df[c_a]

stacked = pd.concat([h_d, a_d]).sort_values(['team', 'date'])
for f in roll_feats:
    if f in stacked.columns:
        stacked[f'roll_{f}'] = stacked.groupby('team')[f].transform(lambda x: x.shift(1).rolling(5, min_periods=1).mean())

df = df.merge(stacked[['match_url', 'team'] + [f'roll_{f}' for f in roll_feats]], left_on=['match_url', 'home_team_name'], right_on=['match_url', 'team'], how='left').drop(columns=['team']).rename(columns={f'roll_{f}': f'home_roll_{f}' for f in roll_feats})
df = df.merge(stacked[['match_url', 'team'] + [f'roll_{f}' for f in roll_feats]], left_on=['match_url', 'away_team_name'], right_on=['match_url', 'team'], how='left').drop(columns=['team']).rename(columns={f'roll_{f}': f'away_roll_{f}' for f in roll_feats})

for f in roll_feats:
    df[f'diff_{f}'] = df[f'home_roll_{f}'] - df[f'away_roll_{f}']

df = df.fillna(0)

# ==========================================
# 3. CONFIGURE MODELS (Hardcoded Winners)
# ==========================================
# Features
features = [
    'diff_elo', 'home_elo', 'away_elo',
    'diff_rest', 'diff_team_points', 'diff_team_xg', 
    'diff_shots_onTarget', 'diff_corners',
    'home_roll_team_xg', 'away_roll_team_xg'
]

# Target
conditions = [
    (df['home_team_score'] > df['away_team_score']),
    (df['home_team_score'] == df['away_team_score']),
    (df['home_team_score'] < df['away_team_score'])
]
y = np.select(conditions, [2, 1, 0])
X = df[features]

print(f"Training on {len(X)} matches using {len(features)} features...")

# --- 1. XGBoost (Tuned) ---
xgb_clf = xgb.XGBClassifier(
    n_estimators=200, learning_rate=0.03, max_depth=3, 
    subsample=0.7, colsample_bytree=0.7, gamma=5,
    objective='multi:softprob', num_class=3, random_state=42
)

# --- 2. Random Forest (Tuned) ---
rf_clf = RandomForestClassifier(
    n_estimators=200, max_depth=5, min_samples_leaf=10, 
    max_features='sqrt', random_state=42
)

# --- 3. Logistic Regression (Tuned & Scaled) ---
lr_clf = make_pipeline(
    StandardScaler(),
    LogisticRegression(C=0.01, solver='lbfgs', multi_class='multinomial', max_iter=1000)
)

# --- 4. The Ensemble (Adjusted Weights) ---
# Previous weights [3,1,2] -> New Weights [2, 1, 5] to favor LR
ensemble = VotingClassifier(
    estimators=[('xgb', xgb_clf), ('rf', rf_clf), ('lr', lr_clf)],
    voting='soft',
    weights=[2, 1, 5] 
)

ensemble.fit(X, y)
print("✅ Ensemble Model Trained.")

# Save Model & Data Helpers
joblib.dump({
    'model': ensemble, 
    'features': features,
    'elo_dict': current_elos,
    'df_recent': df[['date', 'home_team_name', 'away_team_name'] + [c for c in df.columns if 'roll_' in c]].tail(500)
}, 'football_model_final.pkl')

print("✅ Model Saved to 'football_model_final.pkl'")

# ==========================================
# 4. PREDICTION INTERFACE
# ==========================================

def predict_match(home, away):
    print(f"\n🔮 ANALYSIS: {home} vs {away}")
    
    # 1. Get ELOs
    h_elo = current_elos.get(home, 1500)
    a_elo = current_elos.get(away, 1500)
    
    # 2. Get Rolling Stats (Latest available)
    def get_latest(team):
        rows = df[(df['home_team_name']==team) | (df['away_team_name']==team)]
        if len(rows) == 0: return None
        last = rows.sort_values('date').iloc[-1]
        prefix = 'home_' if last['home_team_name']==team else 'away_'
        return {f: last[f"{prefix}roll_{f}"] for f in roll_feats}

    h_stats = get_latest(home)
    a_stats = get_latest(away)
    
    if not h_stats or not a_stats:
        print("❌ Team data not found.")
        return

    # 3. Build Input
    input_data = {
        'diff_elo': (h_elo + 65) - a_elo,
        'home_elo': h_elo,
        'away_elo': a_elo,
        'diff_rest': 0, # Assume equal rest for future prediction
        'home_roll_team_xg': h_stats['team_xg'],
        'away_roll_team_xg': a_stats['team_xg']
    }
    
    # Calc diffs
    for f in roll_feats:
        input_data[f"diff_{f}"] = h_stats[f] - a_stats[f]
        
    # Predict
    input_df = pd.DataFrame([input_data])[features]
    probs = ensemble.predict_proba(input_df)[0]
    
    # Output
    print(f"📊 Win Probabilities:")
    print(f"   🏠 {home}: {probs[2]:.1%} (Odds: {1/probs[2]:.2f})")
    print(f"   🤝 Draw:        {probs[1]:.1%} (Odds: {1/probs[1]:.2f})")
    print(f"   ✈️ {away}: {probs[0]:.1%} (Odds: {1/probs[0]:.2f})")
    
    conf = max(probs)
    winner = "Home" if probs[2]==conf else "Draw" if probs[1]==conf else "Away"
    
    if conf > 0.60: print("💎 SNIPER BET (High Confidence)")
    elif conf > 0.50: print("✅ VALUE BET (Good Confidence)")
    elif conf > 0.45: print("⚠️ RISKY BET (Moderate Confidence)")
    else: print("❌ NO BET (Coin Flip)")

# TEST
predict_match("Real Madrid", "Barcelona")
predict_match("Liverpool", "Manchester City")

   FOOTBALL MATCH PREDICTOR (Advanced Pipeline)   
Loaded 1272 matches from match_data.csv
Parsing compound stats (e.g., '5 of 16')...
Calculating Weighted xG for missing values...
Imputing remaining stats with Team Medians...
Generating Rolling Averages (Window=5)...
Creating Differential Features...
Final Feature Count: 19
Train Size: 1081 | Test Size: 191
Running Grid Search... (This might take 2-3 minutes)
Fitting 3 folds for each of 72 candidates, totalling 216 fits

✅ Best Parameters: {'gamma': 5, 'learning_rate': 0.05, 'max_depth': 3, 'min_child_weight': 3, 'n_estimators': 169, 'subsample': 0.7}
✅ Best Validation Score: 53.09%

🏆 Test Set Accuracy with Tuned Model: 55.50%
   TUNED ENSEMBLE ACCURACY: 55.50%   
Threshold > 0.45: 133 matches | Accuracy: 57.89%
Threshold > 0.50: 111 matches | Accuracy: 55.86%
Threshold > 0.60: 50 matches | Accuracy: 70.00%


## Loading and Using model

In [ ]:
import pandas as pd
import numpy as np
import joblib
from scipy.optimize import minimize
import warnings

warnings.filterwarnings('ignore')

# ==========================================
# 1. LOAD THE ELO MODEL
# ==========================================
print("Loading ELO model...")
try:
    data = joblib.load('football_elo_model.pkl')
    best_model = data['model']
    team_stats = data['stats']
    le = data['encoder']
    print("ELO Model Loaded Successfully!")
except FileNotFoundError:
    print("Error: 'football_elo_model.pkl' not found.")
    exit()

# ==========================================
# 2. FEATURE GENERATION (ELO LOGIC)
# ==========================================
def get_match_features_elo(home_name, away_name, team_stats):
    if home_name not in team_stats or away_name not in team_stats:
        return None, f"One or both teams ({home_name}, {away_name}) not found."

    h = team_stats[home_name]
    a = team_stats[away_name]

    def get_avg(hist, window=5):
        if not hist: return 0
        return sum(hist[-window:]) / min(len(hist), window)

    # Extract Data (using .get() for safety)
    features = {
        'Home_Elo': h.get('elo', 1500),
        'Away_Elo': a.get('elo', 1500),
        'Elo_Diff': h.get('elo', 1500) - a.get('elo', 1500),
        
        'Home_HomeForm': get_avg(h.get('pts_home', [])),
        'Away_AwayForm': get_avg(a.get('pts_away', [])),
        'Specific_Form_Diff': get_avg(h.get('pts_home', [])) - get_avg(a.get('pts_away', [])),
        
        'Home_Gen_XG': get_avg(h.get('xg_all', [])),
        'Away_Gen_XG': get_avg(a.get('xg_all', [])),
        'Home_Gen_Poss': get_avg(h.get('poss_all', [])),
        'Away_Gen_Poss': get_avg(a.get('poss_all', []))
    }
    
    # EXACT column order from training
    cols = [
        'Home_Elo', 'Away_Elo', 'Elo_Diff',
        'Home_HomeForm', 'Away_AwayForm', 'Specific_Form_Diff',
        'Home_Gen_XG', 'Away_Gen_XG',
        'Home_Gen_Poss', 'Away_Gen_Poss'
    ]
    
    return pd.DataFrame([features], columns=cols), None

# ==========================================
# 3. OPTIMIZER ENGINE
# ==========================================
def optimize_stakes(all_bets, bankroll, max_risk, kelly_fraction):
    profitable_bets = []
    
    for bet in all_bets:
        p = bet['Model_Prob']
        odds = bet['Bookie_Odds']
        if odds <= 1: continue
        
        b = odds - 1
        q = 1 - p
        ev = (p * b) - q
        
        # Filter: Positive Edge Only
        if ev > 0.005: 
            profitable_bets.append({
                'Match': bet['Match_ID'],
                'Pick': bet['Pick'],
                'Prob': p,
                'Odds': odds,
                'b': b,
                'EV': ev
            })

    if not profitable_bets:
        return pd.DataFrame(), "No profitable bets found."

    n = len(profitable_bets)
    
    def objective(stakes):
        growth = 0
        for i, bet in enumerate(profitable_bets):
            f = stakes[i]
            if 1 - f <= 0: return np.inf
            growth += bet['Prob'] * np.log(1 + f * bet['b']) + (1 - bet['Prob']) * np.log(1 - f)
        return -growth

    cons = [{'type': 'ineq', 'fun': lambda x: max_risk - np.sum(x)}]
    bounds = [(0, max_risk) for _ in range(n)]
    guess = np.array([max_risk / (n*2)] * n)
    
    result = minimize(objective, guess, method='SLSQP', bounds=bounds, constraints=cons)

    if not result.success: return pd.DataFrame(), "Optimization failed."

    output = []
    optimal_stakes = result.x * kelly_fraction 
    
    for i, bet in enumerate(profitable_bets):
        stake_pct = optimal_stakes[i]
        cash = bankroll * stake_pct
        
        if cash > 1.00:
            fair_odds = 1 / bet['Prob']
            output.append({
                'Match': bet['Match'],
                'Pick': bet['Pick'],
                'Prob': f"{bet['Prob']:.1%}",
                'Real_Odds': bet['Odds'],
                'Fair_Odds': f"{fair_odds:.2f}",
                'Edge': f"{bet['EV']:.1%}",
                'Stake_%': f"{stake_pct:.2%}",
                'Cash': f"${cash:.2f}"
            })
            
    return pd.DataFrame(output), None

# ==========================================
# 4. EXECUTION WITH REAL ODDS
# ==========================================

# SETTINGS
BANKROLL = 1000  # Set your real bankroll here
MAX_RISK = 0.25   # Max 25% of bankroll for this entire batch
KELLY = 0.25      # Quarter Kelly Safety

# DATA EXTRACTED FROM YOUR IMAGE
upcoming_fixtures_with_odds = [
    # Today / Tonight
    {'Home': 'Tottenham Hotspur', 'Away': 'Liverpool', 'Home_Odds': 3.54, 'Draw_Odds': 3.60, 'Away_Odds': 2.08},
    {'Home': 'Everton', 'Away': 'Arsenal', 'Home_Odds': 6.17, 'Draw_Odds': 3.94, 'Away_Odds': 1.59},
    {'Home': 'Leeds United', 'Away': 'Crystal Palace', 'Home_Odds': 2.72, 'Draw_Odds': 3.19, 'Away_Odds': 2.76},
    
    # Tomorrow / Dec 22
    {'Home': 'Aston Villa', 'Away': 'Manchester United', 'Home_Odds': 2.11, 'Draw_Odds': 3.67, 'Away_Odds': 3.35},
    {'Home': 'Fulham', 'Away': 'Nottingham Forest', 'Home_Odds': 2.47, 'Draw_Odds': 3.26, 'Away_Odds': 2.99},
    
    # Boxing Day / Late Dec
    {'Home': 'Manchester United', 'Away': 'Newcastle United', 'Home_Odds': 2.12, 'Draw_Odds': 3.78, 'Away_Odds': 3.19},
    {'Home': 'Nottingham Forest', 'Away': 'Manchester City', 'Home_Odds': 4.91, 'Draw_Odds': 4.17, 'Away_Odds': 1.65},
    {'Home': 'Arsenal', 'Away': 'Brighton & Hove Albion', 'Home_Odds': 1.45, 'Draw_Odds': 4.61, 'Away_Odds': 7.12},
    {'Home': 'Brentford', 'Away': 'Bournemouth', 'Home_Odds': 2.26, 'Draw_Odds': 3.62, 'Away_Odds': 3.03},
    {'Home': 'Burnley', 'Away': 'Everton', 'Home_Odds': 4.14, 'Draw_Odds': 3.41, 'Away_Odds': 1.95},
    {'Home': 'Liverpool', 'Away': 'Wolverhampton Wanderers', 'Home_Odds': 1.22, 'Draw_Odds': 6.77, 'Away_Odds': 12.06},
    {'Home': 'West Ham United', 'Away': 'Fulham', 'Home_Odds': 2.70, 'Draw_Odds': 3.39, 'Away_Odds': 2.61},
    {'Home': 'Chelsea', 'Away': 'Aston Villa', 'Home_Odds': 1.84, 'Draw_Odds': 3.80, 'Away_Odds': 4.05},
    {'Home': 'Sunderland', 'Away': 'Leeds United', 'Home_Odds': 2.56, 'Draw_Odds': 3.33, 'Away_Odds': 2.77},
    {'Home': 'Crystal Palace', 'Away': 'Tottenham Hotspur', 'Home_Odds': 2.15, 'Draw_Odds': 3.42, 'Away_Odds': 3.36},
    {'Home': 'Manchester United', 'Away': 'Wolverhampton Wanderers', 'Home_Odds': 1.29, 'Draw_Odds': 5.90, 'Away_Odds': 9.80}
]

print("\nScanning market for value using ELO Model...")
all_bets = []

for f in upcoming_fixtures_with_odds:
    home, away = f['Home'], f['Away']
    match_id = f"{home} vs {away}"
    
    # Get Features
    feat, err = get_match_features_elo(home, away, team_stats)
    
    if err:
        # Try finding partial name matches if exact name fails
        print(f"Skipping {match_id}: {err}")
        continue
        
    probs = best_model.predict_proba(feat)[0] # [Away, Draw, Home]
    
    # Add potential bets
    all_bets.append({'Match_ID': match_id, 'Pick': 'Home', 'Model_Prob': probs[2], 'Bookie_Odds': f['Home_Odds']})
    all_bets.append({'Match_ID': match_id, 'Pick': 'Draw', 'Model_Prob': probs[1], 'Bookie_Odds': f['Draw_Odds']})
    all_bets.append({'Match_ID': match_id, 'Pick': 'Away', 'Model_Prob': probs[0], 'Bookie_Odds': f['Away_Odds']})

# Run Optimization
df_results, msg = optimize_stakes(all_bets, BANKROLL, MAX_RISK, KELLY)

print("\n" + "="*85)
print(f"OPTIMIZED ELO PORTFOLIO (Bankroll: ${BANKROLL})")
print("="*85)

if not df_results.empty:
    print(df_results.to_string(index=False))
    total = df_results['Cash'].str.replace('$','').astype(float).sum()
    print("-" * 85)
    print(f"TOTAL INVESTMENT: ${total:.2f} ({total/BANKROLL:.1%} of Bankroll)")
else:
    print(msg)

Loading ELO model...
ELO Model Loaded Successfully!

Scanning market for value using ELO Model...

OPTIMIZED ELO PORTFOLIO (Bankroll: $1000)
                                       Match Pick  Prob  Real_Odds Fair_Odds   Edge Stake_%   Cash
                          Everton vs Arsenal Home 28.0%       6.17      3.57  72.7%   0.53%  $5.26
        Nottingham Forest vs Manchester City Home 33.1%       4.91      3.02  62.7%   0.26%  $2.60
                          Burnley vs Everton Home 46.1%       4.14      2.17  90.9%   2.02% $20.16
        Liverpool vs Wolverhampton Wanderers Away 25.8%      12.06      3.88 210.6%   2.37% $23.72
Manchester United vs Wolverhampton Wanderers Away 21.7%       9.80      4.61 112.4%   1.06% $10.63
-------------------------------------------------------------------------------------
TOTAL INVESTMENT: $62.37 (6.2% of Bankroll)


In [48]:
import pandas as pd
import joblib

# Load the ELO Stats
# (Assuming 'team_stats' is loaded. If not, uncomment the lines below)
# data = joblib.load('football_elo_model.pkl')
# team_stats = data['stats']

def check_matchup_stats_elo(home, away):
    if home not in team_stats or away not in team_stats:
        print("One or both teams not found.")
        return

    h = team_stats[home]
    a = team_stats[away]
    
    # Helper to safe-get averages
    def get_avg(hist): return sum(hist[-5:]) / 5 if hist else 0
    def get_val(stats, key): return stats.get(key, 0)

    print(f"\n{'STATISTIC':<20} | {home.upper():<15} | {away.upper():<15} | {'EDGE'}")
    print("-" * 65)
    
    # 1. ELO RATING (The most important stat)
    h_elo = h.get('elo', 1500)
    a_elo = a.get('elo', 1500)
    print(f"{'ELO Rating':<20} | {str(round(h_elo, 0)):<15} | {str(round(a_elo, 0)):<15} | {'Home' if h_elo > a_elo else 'Away'}")

    # 2. FORM (Last 5 Games Points - using 'pts_all')
    h_form = get_avg(h.get('pts_all', []))
    a_form = get_avg(a.get('pts_all', []))
    print(f"{'Recent Form (Pts)':<20} | {h_form:<15.2f} | {a_form:<15.2f} | {'Home' if h_form > a_form else 'Away'}")

    # 3. ATTACK (Average xG - using 'xg_all')
    h_xg = get_avg(h.get('xg_all', []))
    a_xg = get_avg(a.get('xg_all', []))
    print(f"{'Attack (xG)':<20} | {h_xg:<15.2f} | {a_xg:<15.2f} | {'Home' if h_xg > a_xg else 'Away'}")

    # 4. POSSESSION (using 'poss_all')
    h_poss = get_avg(h.get('poss_all', []))
    a_poss = get_avg(a.get('poss_all', []))
    print(f"{'Possession':<20} | {h_poss:<15.1%} | {a_poss:<15.1%} | {'Home' if h_poss > a_poss else 'Away'}")

# Run it for the big anomaly
for upcoming_fixture in upcoming_fixtures_with_odds:
    check_matchup_stats_elo(upcoming_fixture["Home"],upcoming_fixture["Away"])


STATISTIC            | TOTTENHAM HOTSPUR | LIVERPOOL       | EDGE
-----------------------------------------------------------------
ELO Rating           | 1518.0          | 1547.0          | Away
Recent Form (Pts)    | 2.00            | 2.00            | Away
Attack (xG)          | 1.48            | 1.90            | Away
Possession           | 51.2%           | 53.0%           | Away

STATISTIC            | EVERTON         | ARSENAL         | EDGE
-----------------------------------------------------------------
ELO Rating           | 1538.0          | 1666.0          | Away
Recent Form (Pts)    | 2.60            | 1.60            | Home
Attack (xG)          | 1.32            | 1.74            | Away
Possession           | 42.0%           | 62.6%           | Away

STATISTIC            | LEEDS UNITED    | CRYSTAL PALACE  | EDGE
-----------------------------------------------------------------
ELO Rating           | 1464.0          | 1574.0          | Away
Recent Form (Pts)    | 1.40  

In [ ]:
import pandas as pd
import numpy as np
import joblib

# ==========================================
# 1. LOAD THE SAVED MODEL
# ==========================================
print("Loading model...")
data = joblib.load('football_model_optimized.pkl')

best_model = data['model']
team_stats = data['team_stats']
le = data['encoder']

print("Model Loaded Successfully!")

# ==========================================
# 2. PREDICTION FUNCTION
# ==========================================
def predict_weekend(mananger,matchups, model, team_stats, encoder):
    print(f"\n{'MATCHUP':<40} | {'HOME %':<7} | {'DRAW %':<7} | {'AWAY %':<7} | {'PREDICTION':<12} | {'ACTION'}")
    print("-" * 105)
    
    # Helper to calculate stats exactly like training
    def get_avg(hist, window=5):
        if not hist: return 0
        return sum(hist[-window:]) / min(len(hist), window)

    for home, away in matchups:
        # Check data exists
        if home not in team_stats or away not in team_stats:
            print(f"{home:<18} vs {away:<18} | ???     | ???     | ???     | N/A          | ❌ Unknown Team")
            continue
            
        h = team_stats[home]
        a = team_stats[away]
        
        # Build Features
        features = {
            'HomeTeam_ID': encoder.transform([home])[0],
            'AwayTeam_ID': encoder.transform([away])[0],
            'Home_Form_Pts': get_avg(h['pts']),
            'Away_Form_Pts': get_avg(a['pts']),
            'Form_Diff': get_avg(h['pts']) - get_avg(a['pts']),
            'Home_xG_Avg': get_avg(h['xg']),
            'Away_xG_Avg': get_avg(a['xg']),
            'xG_Diff': get_avg(h['xg']) - get_avg(a['xg']),
            'Home_xGA_Avg': get_avg(h['xga']),
            'Away_xGA_Avg': get_avg(a['xga']),
            'Def_Diff': get_avg(h['xga']) - get_avg(a['xga']),
            'Home_Poss_Avg': get_avg(h['poss']),
            'Away_Poss_Avg': get_avg(a['poss']),
            'Poss_Diff': get_avg(h['poss']) - get_avg(a['poss'])
        }
        
        row = pd.DataFrame([features])
        
        # Predict
        probs = model.predict_proba(row)[0]
        p_away, p_draw, p_home = probs[0], probs[1], probs[2]
        odds_away, odds_draw, odds_home = 1/probs[0], 1/probs[1], 1/probs[2]
        confidence = np.max(probs)
        pred_label = ['Away Win', 'Draw', 'Home Win'][np.argmax(probs)]
        
        # --- ACTION LOGIC (Based on your 72% Accuracy) ---
        action = "-"
        if confidence > 0.50: action = "Low Bet"
        if confidence > 0.55: action = "✅ BET"         # 65% Accuracy Zone
        if confidence > 0.60: action = "🔥 SNIPER BET"  # 72% Accuracy Zone
        
        if pred_label != 'Draw' and p_draw > 0.32: action += " (⚠️ Risky)"
        manager.calculate_bet(odds=odds_home, model_confidence=confidence, historic_accuracy=0.725)

        print(f"{home:<18} vs {away:<18} | {p_home:.1%} {odds_home:.1f}   | {p_draw:.1%} {odds_draw:.1f}   | {p_away:.1%} {odds_away:.1f}   | {pred_label:<12} | {action}")

# ==========================================
# 3. REAL FIXTURES (DEC 20-21, 2025)
# ==========================================
# Update this list with the matches you want to check today
weekend_games = [
    # Premier League
    ('Tottenham Hotspur', 'Liverpool'),
    ('Manchester City', 'West Ham United'),
    ('Aston Villa', 'Manchester United'),
    ('Everton', 'Arsenal'),
    ('Newcastle United', 'Chelsea'),
    
    # La Liga
    ('Real Madrid', 'Sevilla'),
    ('Villarreal', 'Barcelona'),
    ('Girona', 'Atlético Madrid'),
    
    # Serie A
    ('Juventus', 'Roma'),
    ('Napoli', 'Lazio')
]

# Run
predict_weekend(manager,weekend_games, best_model, team_stats, le)

Loading model...
Model Loaded Successfully!

MATCHUP                                  | HOME %  | DRAW %  | AWAY %  | PREDICTION   | ACTION
---------------------------------------------------------------------------------------------------------
Tottenham Hotspur  vs Liverpool          | 41.6% 2.4   | 25.4% 3.9   | 32.9% 3.0   | Home Win     | -
Manchester City    vs West Ham United    | 60.8% 1.6   | 22.9% 4.4   | 16.3% 6.1   | Home Win     | 🔥 SNIPER BET
Aston Villa        vs Manchester United  | 52.7% 1.9   | 22.0% 4.5   | 25.3% 3.9   | Home Win     | Low Bet
Everton            vs Arsenal            | 35.6% 2.8   | 20.8% 4.8   | 43.6% 2.3   | Away Win     | -
Newcastle United   vs Chelsea            | 37.8% 2.6   | 26.9% 3.7   | 35.3% 2.8   | Home Win     | -
Real Madrid        vs Sevilla            | 40.2% 2.5   | 26.6% 3.8   | 33.2% 3.0   | Home Win     | -
Villarreal         vs Barcelona          | 24.2% 4.1   | 20.7% 4.8   | 55.2% 1.8   | Away Win     | ✅ BET
Girona             

In [ ]:
def predict_weekend(matchups, model, team_stats, encoder):
